In [ ]:
import requests
import hmac
import json
import base64
import hashlib
import pandas as pd
import datetime, time
from time import sleep

In [ ]:
base_url = "https://api.sandbox.gemini.com"
endpoint = "/v1/order/new"
url = base_url + endpoint

In [ ]:
gemini_api_key = "master-2rJeGEcrQbx6OKXvSvLR"
gemini_api_secret = "CZTk6cmFZgVPSnoHij22LZNFZ3S".encode()

In [ ]:
def Auth(payload):
    
    encoded_payload = json.dumps(payload).encode()
    b64 = base64.b64encode(encoded_payload)
    signature = hmac.new(gemini_api_secret, b64, hashlib.sha384).hexdigest()

    request_headers = { 'Content-Type': "text/plain",
                'Content-Length': "0",
                'X-GEMINI-APIKEY': gemini_api_key,
                'X-GEMINI-PAYLOAD': b64,
                'X-GEMINI-SIGNATURE': signature,
                'Cache-Control': "no-cache" }
    return(request_headers)

In [ ]:
while True:
    try:
        response = requests.get("https://api.gemini.com/v1/pubticker/BTCUSD").json()
        btc_old = float(response['ask'])
        print(btc_old)
    except exception as e:
        print('Error obtaining data')
    
    sleep(300)
    
    try:
        response = requests.get("https://api.gemini.com/v1/pubticker/BTCUSD").json()
        btc_new = float(response['ask'])
        print(btc_new)
    except exception as e:
        print('Error obtaining data')
    
    percent = ((btc_new - btc_old)*100) / btc_old
        
    if percent >= 5:
        try:
            response = requests.get("https://api.gemini.com/v1/pubticker/ETHUSD").json()
            eth_price = float(response['ask'])
            print(eth_price)
        except Exception as e:
            print('Error obtaining data')
            
        t = datetime.datetime.now()
        payload_nonce =  str(int(time.mktime(t.timetuple())*1000))  
        
        payload = {
            "request": "/v1/order/new",
            "nonce": payload_nonce,
            "symbol": "ethusd",
            "amount": "1",
            "price": eth_price,
            "side": "buy",
            "type": "exchange limit",
            "account": "algotrading"
            }
        
        
        try:
            new_order = requests.post(url,
                            data=None,
                            headers=Auth(payload)).json()
            print(new_order)
            
        except Exception as e:
            print('Error placing order')
            
        print(new_order)

        check_status = new_order['order_id']

        sleep(2)

        t = datetime.datetime.now()
        payload_nonce =  str(int(time.mktime(t.timetuple())*1000))

        payload1 = {
            "request" : '/v1/order/status',
            "nonce" : payload_nonce,
            "order_id" : check_status,
            "account" : "algotrading", 
        }
        
        try:  
            check = requests.post(base_url + '/v1/order/status',
                            data=None,
                            headers=Auth(payload1)).json()
            
        except Exception as e:
            print('Error checking order')

        if str(check['is_live']) == 'True':
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break

        else:
            print('Order was canceled {}'.format(pd.Timestamp.now()))
            break

    else:
        print(f'BTC did not reach the requirement. Percentage move is at {percent}')
        continue